<a href="https://colab.research.google.com/github/abhi0618/BERT-CONLL03/blob/main/BERT_Conll_with_pretrained_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q  transformers

In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import pandas as pd 
import spacy


MODEL_NAME = "dslim/bert-base-NER"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [7]:
ents = []
sentence = "This paper examines the shareholder primacy norm (SPN) as a widely acknowledged impediment to corporate social responsibility and explores the role of business schools in promoting the SPN but also potentially as an avenue for change by addressing misconceptions about shareholder primacy and the purpose of business. We start by explaining the SPN and then review its status under US and UK laws and show that it is not a likely legal requirement, at least under the guise of shareholder value maximization. This is in contrast to the common assertion that managers are legally constrained from addressing CSR issues if doing so is inconsistent with the economic interests of shareholders. Nonetheless, while the SPN might be muted as a legal norm, we show that it is certainly evident as a social norm among managers and in business schools—reflective, in part, of the sole voting rights of shareholders on corporate boards and of the dominance of shareholder theory—and justifiably so in the view of many managers and business academics. We argue that this view is misguided, not least when associated with claims of a purported legally enforceable requirement to maximize shareholder value. We propose two ways by which the influence of the SPN among managers might be attenuated: extending fiduciary duties of executives to non-shareholder stakeholders and changes in business school teaching such that it covers a plurality of conceptions of the purpose of the corporation. ©️ 2014, Springer Science+Business Media Dordrecht."
ner_results = nlp(sentence)
df = pd.DataFrame(ner_results)

for i in range(len(df)):
    ents.append({
                'start': df["start"][i], 
                'end' : df["end"][i],
                "label" : df["entity"][i]})
    
doc = {
    'text' : sentence,
    "ents" : ents
}
colors = {"Annotation" :"linear-gradient(90deg, #aa9cfc, #fc9ce7)" } 
options = {"colors": colors}
spacy.displacy.render(doc, style="ent", options = options , manual=True, jupyter=True);

In [8]:
df

,entity,score,index,word,start,end
0,B-ORG,0.851349,35,SP,185,187
1,I-ORG,0.521605,36,##N,187,188
2,B-ORG,0.888396,68,SP,345,347
3,B-LOC,0.959768,76,US,382,384
4,B-LOC,0.996234,78,UK,389,391
5,B-ORG,0.899311,141,SP,714,716
6,I-ORG,0.612441,142,##N,716,717
7,B-ORG,0.914268,250,SP,1245,1247
8,I-ORG,0.592499,251,##N,1247,1248
9,B-ORG,0.999146,299,Springer,1489,1497


In [10]:
def metric_fn(label_ids, pred_ids, per_example_loss, input_mask):
                    # ['<pad>'] + ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC", "X"]
                    indices = [2, 3, 4, 5, 6, 7, 8, 9]
                    precision = tf_metrics.precision(label_ids, pred_ids, num_labels, indices, input_mask)
                    recall = tf_metrics.recall(label_ids, pred_ids, num_labels, indices, input_mask)
                    f = tf_metrics.f1(label_ids, pred_ids, num_labels, indices, input_mask)
                    accuracy = tf.metrics.accuracy(label_ids, pred_ids, input_mask)
                    loss = tf.metrics.mean(per_example_loss)
                    return {
                        'eval_precision': precision,
                        'eval_recall': recall,
                        'eval_f': f,
                        'eval_accuracy': accuracy,
                        'eval_loss': loss,
                    }

              return eval_precision   ;

IndentationError: ignored

In [ ]:
!pip install evaluate
import evaluate
from datasets import load_dataset
from evaluate import evaluator
from transformers import AutoModelForSequenceClassification, pipeline
data = load_dataset("conll2003", split="test").shuffle(seed=42).select(range(1000))

task_evaluator = evaluator("text-classification")
eval_results = task_evaluator.compute(
    model_or_pipeline="dslim/bert-base-NER",
    data=data,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
)
print(eval_results)
